In [1]:
## imports
%matplotlib qt


import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import matplotlib
from matplotlib import cm
import h5py
import os
from math import pi, sqrt
from scipy.optimize import curve_fit
import scipy
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import time
from scipy.interpolate import InterpolatedUnivariateSpline
rc('text', usetex=True)
import seaborn as sns
from os import path
from scipy import integrate

# Plot parameters
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['axes.labelsize'] = 24
matplotlib.rcParams['xtick.labelsize'] = 30
matplotlib.rcParams['ytick.labelsize'] = 30
matplotlib.rcParams['xtick.major.size'] = 20
matplotlib.rcParams['ytick.major.size'] = 20
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['xtick.minor.size'] = 10
matplotlib.rcParams['ytick.minor.size'] = 10
matplotlib.rcParams['legend.fontsize'] = 18
matplotlib.rcParams['legend.frameon'] = True
matplotlib.rcParams['lines.linewidth'] = 3


In [2]:
## Data helper functions

def CutTimes(time, data, TLow, TUp): 
    TLowIndex = np.where(time <= TLow)[0][0]
    TUpIndex = np.where(time >= TUp)[0][-1]
    time = time[TLowIndex:TUpIndex]
    data = data[TLowIndex:TUpIndex]
    return time, data

def CutTimesArray(time, Arr, TLow, TUp): 
    TLowIndex = np.where(time <= TLow)[0] 
    if(len(TLowIndex) > 0):
        TLowIndex = TLowIndex[-1]
    else:
        TLowIndex = 0
    TUpIndex = np.where(time >= TUp)[0]
    if(len(TUpIndex) > 0):
        TUpIndex = TUpIndex[0]
    else:
        TUpIndex = -1
        
    time = time[TLowIndex:TUpIndex]
    Ans = []
    for data in Arr:
        Ans.append(data[TLowIndex:TUpIndex])
    return time, Ans 


def GetPeakTimeMode(time, data): ###
    ## Peak time being the peak of the magnitude of the data
    t_peak = time[np.argmax(np.absolute(data))]
    return t_peak

def GetPeakTimeModeReal(time, data): ###
    ## Peak time being the peak of the magnitude of the data
    t_peak = time[np.argmax(np.real(data))]
    return t_peak

def SubtractPeakTimeMode(time, data): ###
    t_peak = GetPeakTimeMode(time, data)
    return time - t_peak

def SubtractPeakTimeModeReal(time, data): ###
    t_peak = GetPeakTimeModeReal(time, data)
    return time - t_peak

def InterpolateTimes(time, data, time_dest):
    """ Interpolates time, data onto new time axis
        time_dest """
    interpolant = scipy.interpolate.CubicSpline(time, data)
    return interpolant(time_dest)

def GetCameraPosition(p):
    p = p.split('Trace')[1]
    x = p.split('_')[1]
    y = p.split('_')[2]
    z = p.split('_')[3]
    return '[' + x + ',' + y + ',' + z + ']'

def GetTime(p):
    p = p.split('Trace')[1]
    t = p.split('_')[4]
    t = t.split('/')[0]
    return t

## Read in horizon data 

In [3]:
## Read in horizon trajectories

Lev = "2"

def read_horizon_trajectories(Horizon):
    """ Horizon is a string corresponding to the BH we're interested in [A, B, C]"""
    f = "Data/HeadOn_Harmonic/JoinedLev" + Lev + "/ApparentHorizons/Trajectory_Ah" + Horizon + ".dat"
    t, x, y, z = np.loadtxt(f, comments="#",usecols=([0,1,2,3]),unpack=True)
    return t, x, y, z

## Build the horizon interpolants
t_a, x_a, y_a, z_a = read_horizon_trajectories("A")
spl_x_a = InterpolatedUnivariateSpline(t_a, x_a)
spl_y_a = InterpolatedUnivariateSpline(t_a, y_a)
spl_z_a = InterpolatedUnivariateSpline(t_a, z_a)

t_b, x_b, y_b, z_b = read_horizon_trajectories("B")
spl_x_b = InterpolatedUnivariateSpline(t_b, x_b)
spl_y_b = InterpolatedUnivariateSpline(t_b, y_b)
spl_z_b = InterpolatedUnivariateSpline(t_b, z_b)

t_c, x_c, y_c, z_c = read_horizon_trajectories("C")
t_merger = t_b[-1]
t_ringdown = t_c[0]
spl_x_c = InterpolatedUnivariateSpline(t_c, x_c)
spl_y_c = InterpolatedUnivariateSpline(t_c, y_c)
spl_z_c = InterpolatedUnivariateSpline(t_c, z_c)

def horizon_at_time(time, Horizon):
    if Horizon == "A":
        return spl_x_a(time), spl_y_a(time), spl_z_a(time)
    if Horizon == "B":
        return spl_x_b(time), spl_y_b(time), spl_z_b(time)
    if Horizon == "C":
        return spl_x_c(time), spl_y_c(time), spl_z_c(time)
    else:
        print("Unrecognized horizon argument")

def distance_sqr(xx, yy, zz, xx_h, yy_h, zz_h):
    return (xx - xx_h)**2 + (yy - yy_h)**2 + (zz - zz_h)**2
    
def distance_to_horizon(time, xx, yy, zz):
    """ time, x,x yy, zz are scalars for the position"""
    if time <= t_merger:
        min_dist = 1e10
        for horizon in ["A", "B"]:
            """ Have to minimize over AhA and AhB distances"""
            xx_h, yy_h, zz_h = horizon_at_time(time, horizon)
            dist = distance_sqr(xx, yy, zz, xx_h, xx_h, xx_h)
            min_dist = min(min_dist, dist)
        return sqrt(min_dist)
    elif time > t_merger:
        xx_h, yy_h, zz_h = horizon_at_time(time, "C")
        dist = distance_sqr(xx, yy, zz, xx_h, xx_h, xx_h)
        return sqrt(dist)
    else:
        print("somehow here")
        print(time, t_merger, t_ringdown)
                
def min_distance_to_horizon(t, x, y, z):
    """ Returns the minimum distance of a geodesic to a horizon over all time
        t, x, y, z are arrays with the history of the geodesic"""
    min_distance = 1e10
                
    ## minimize over the times
    for time, xx, yy, zz in zip(t, x, y, z):
        dist_horizon = distance_to_horizon(time, xx, yy, zz)
        min_distance = min(min_distance, dist_horizon)

    return(sqrt(min_distance))
            

## Read in lensing camera data

In [4]:
## Lensing camera data
def GetCameraData(p):
    
    file = p + 'RefinementMethodData.h5'
    f = h5py.File(file, 'r')
    data = f['LensingCore.dat']
    
    tags = data[:,0]
    tags = tags.astype(int)
    x_pos = data[:,1]
    y_pos = data[:,2]
    surface = data[:,4]
    
    return tags, x_pos, y_pos, surface

def GrabSurfaceIndices(surface, fate):
    
    i = np.where(surface == fate)[0]
    return i

def SelectCameraGeodesics(p, fate):
    
    tags, x_pos, y_pos, surface = GetCameraData(p)
    
    i = np.where(surface == fate)[0]
    
    return tags[i], x_pos[i], y_pos[i], surface[i]
    
def GetInfinityGeodesics(p):
    """ 
    inf = np.where(surface == 1.)[0]
    infty = np.where(surface == 7.)[0]
    aha = np.where(surface == 2.)[0]
    ahb = np.where(surface == 3.)[0]
    ahc = np.where(surface == 4.)[0]
    """
    file = p + 'RefinementMethodData.h5'
    f = h5py.File(file, 'r')
    data = f['LensingCore.dat']
    surface = data[:,4]
    #return GrabSurfaceIndices(surface, 1)
    return GrabSurfaceIndices(surface, 7)
    


## Read in geodesics trajectories

In [5]:
## Read geodesic trajectories from Node0.h5 and dump to .dat files

def ReadGeodesicData(p, t_start, t_end):
    """ Read in an array of times and positions for all geodesics at once, 
        and return the trajectories indexed by geodesic """
        
    def AppendGeodesicsTime(Lev, Run):

        print("Reading Geodesic data for " + Lev + " and " + Run)

        file = p + '/' + Lev + '/' + Run + '/Run/Node0.h5'
        f = h5py.File(file, 'r')
        ## grab the .dat files
        keys = [k for k in f.keys() if 'dat' in k]
        ## Array of times from the .dat files
        times = [float(k.split('.dat')[0]) for k in keys]
        ## sort keys according to times
        times, keys = zip(*sorted(zip(times, keys)))
        # grab the number of geodesics
        N_geodesics = len(f[keys[-1]][:,0])
        print("Total geodesics: ", N_geodesics, "Time steps: ", len(times))
        ## Minimum index
        m = int(f[keys[-1]][:,0][0])
        print("Geodesic index offset of this refinement iteration: ", m)
    
        X = [ [] for _ in range(N_geodesics)]
        Y = [ [] for _ in range(N_geodesics)]
        Z = [ [] for _ in range(N_geodesics)]
        L = [ [] for _ in range(N_geodesics)]
        T = [ [] for _ in range(N_geodesics)]
    
        for k, t in zip(keys, times):
            print(t)
            if ((t > t_start) and (t < t_end)):
                print("%.1f  " % t, end = '')
                data = f[k]
                ## indices and positions for all geodesics at this time
                indices = data[:,0]
                if (len(data[0]) == 7):
                    """ If we've run the numerical evolution so that we haven't 
                        evolved lapse p0 -- just set lapsep0 to 1 in the files. 
                        This field need to be in the resulting .dat file for the 
                        paraview trajectory visualization"""
                    x = data[:,4]
                    y = data[:,5]
                    z = data[:,6]
                    l = np.ones(len(x))
                elif (len(data[0]) == 8):
                    """ If we have evolved lapse p0"""
                    l = data[:,1]
                    x = data[:,5]
                    y = data[:,6]
                    z = data[:,7]
                else:
                    print("Unrecognized number of variables")
                ## fill in the array for each index
                for i, j in zip(indices.astype(int), range(len(indices))):
                    X[i-m] = np.append(X[i-m], x[j])
                    Y[i-m] = np.append(Y[i-m], y[j])
                    Z[i-m] = np.append(Z[i-m], z[j])
                    L[i-m] = np.append(L[i-m], l[j])
                    T[i-m] = np.append(T[i-m], t)

        print("\n")
        ## Once we have the arrays constructed, append them to the file
        print('Read the geodesic data, now writing the files')
        print("Masha", len(T))
        for a in range(len(T)):
            ##if (len(T[a]) > 1):
                ## Remember to add in the minimum index since the starting 
                ## geodesic index just gets incremented during reach refinement
                ## level (by the number of geodesics that came from the levels before)
            ff = open(p + '/Trajectories/' + str(a + m) + '.dat','ab')
            np.savetxt(ff, np.c_[T[a],X[a],Y[a],Z[a],L[a]])
            ff.close()
        print('Finished writing the files')
            
    ## Go through the refinement levels and the segments
    RefinementLevs = [el for el in os.listdir(p) if "Lev" in el]
    print("RefinementLevs:", RefinementLevs)
    for lev in RefinementLevs:
        Segments = os.listdir(p + '/' + lev)
        print(lev + " Segments:", Segments)
        for segment in Segments:
            AppendGeodesicsTime(lev, segment)

def MakeGeodesicDatFiles(p, t_start, t_end):
    """ Print the result of ReadGeodesicData to files """
    ReadGeodesicData(p, t_start, t_end)

## Functions for reading GetTrajectoriesFromH5 output
def GetGeodesicTrajectory(p, n, Culled = False):
    """ Read in the post-processed trajectory for the nth geodesic """
    f = p + 'Trajectories/' + str(n) + '.dat'
    if Culled:
        f = p + 'CulledTrajectories/' + str(n) + '.dat'
    #t, x, y, z, lapse = np.loadtxt(f, comments="#",usecols=([0,5,6,7,1]),unpack=True)
    t, x, y, z, lapse = np.loadtxt(f, comments="#",usecols=([0,1,2,3,4]),unpack=True)
    return t, x, y, z, lapse

def GetGeodesicIndices(p, Culled = False):
    """ Return the indices of all of the geodesics we have printed to file """
    Files = os.listdir(p + '/Trajectories')
    if Culled:
        Files = os.listdir(p + '/CulledTrajectories')
    Indices = [int(file.split('.dat')[0]) for file in Files]
    Indices = sorted(Indices)
    return Indices

In [ ]:
##MakeGeodesicDatFiles('Data/TraceHeadOn_0_0_100_500_BohnNumerical', 330, 1000)

## Zero crossings computations

In [22]:
## Zero crossings computations
def ComputeZeroCrossings(p, n):
    """ Compute the number of zero crossings in the y-z plane - 
        useful for head-on runs where the collision happens
        along the x axis """
    t, x, y, z, lapse = GetGeodesicTrajectory(p, n)
    theta = np.arctan2(z, y)
    zero_crossings = len(np.where(np.diff(np.sign(theta)))[0])
    return zero_crossings

def MakeZeroCrossingsFile(p):
    """ Make a file with the format [geodesic index, number of zero crossings] so 
        that we only have to compute the number of zero crossings once """
    ns = GetGeodesicIndices(p)
    crossings = [ComputeZeroCrossings(p, n) for n in ns]
    np.savetxt(p + 'ZeroCrossings.dat', np.c_[ns, crossings], fmt = '%d %d')
    
def GetGeodesicsZeroCrossings(p):
    """ Return the zero crossings for each index """
    f = p + 'ZeroCrossings.dat'
    ns, zero_crossings = np.loadtxt(f, comments="#",usecols=([0,1]),unpack=True,dtype=int)
    return ns, zero_crossings

def GetGeodesicsZeroCrossingsIndices(p, N):
    """ Return the indices of the geodesics that make N zero-crossings """
    f = p + 'ZeroCrossings.dat'
    ns, zero_crossings = np.loadtxt(f, comments="#",usecols=([0,1]),unpack=True,dtype=int)
    indices = ns[np.where(zero_crossings == N)[0]]
    return indices

def GetGeodesicsMaxZeroCrossings(p):
    """ Return the maximum number N of zero crossings for a given run, 
        along with the indices of the geodesics that make N zero-crossings """
    f = p + 'ZeroCrossings.dat'
    ns, zero_crossings = np.loadtxt(f, comments="#",usecols=([0,1]),unpack=True,dtype=int)
    max_N = max(zero_crossings)
    indices = ns[np.where(zero_crossings == max_N)[0]]
    return max_N, indices
    


In [80]:
##MakeZeroCrossingsFile('Data/TraceHeadOn_0_0_100_319.7_quick/')
#GetGeodesicsZeroCrossingsIndices('Data/TraceHeadOn_0_0_100_319.7/', 3)

## Plot geodesic trajectories

In [23]:
## Plot geodesic trajectories
def PlotGetTrajectoriesFromH5(p):

    max_N, Indices = GetGeodesicsMaxZeroCrossings(p)
    print(max_N, len(Indices))
    
    #Indices = [75256]
    
    cs = sns.color_palette('Paired', n_colors=len(Indices))
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')

    for n, color in zip(Indices, cs):
        t, x, y, z, lapse = GetGeodesicTrajectory(p, n)
        #t, Arr = CutTimesArray(t, [x, y, z, lapse], 140, 180)
        #x, y, z, lapse = Arr
        ax.plot(x, y, z, lw = 2.0, label = n, color = color)
        #ss = ax.scatter(x, y, z, s=20, c=lapse, cmap = 'rainbow', vmin=0, vmax=12)
        #skip = 10
        #for tt, xx, yy, zz in zip(t[::skip], x[::skip], y[::skip], z[::skip]):
        #    ax.text(xx, yy, zz, str(tt))
    #cbar = fig.colorbar(ss, fraction=0.03, pad=0.04,  orientation="horizontal")
    #cbar.set_label(r'$\log \alpha p_0$', rotation=0)
                
    
    
    #Add in the horizons
    for horizon, color in zip(["A", "B", "C"], ["blue", "lightblue", "yellow"]):
        t_h, x_h, y_h, z_h = read_horizon_trajectories(horizon)
        #t_h, x_h, y_h, z_h, temp = CutTimesGeodesic(t_h, x_h, y_h, z_h, z_h, 155, 180)
        plt.plot(x_h, y_h, z_h, color='black', lw = 2.0, ls='-')
    #ax.scatter([0.0], [0.0], [0.0], s = 20, color='black')  
    
    ax.set_xlabel('x/M',labelpad=20)
    ax.set_ylabel('y/M', labelpad=20)
    ax.set_zlabel('z/M', labelpad=20)
    
    lim = 10.0
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)
    
    ax.set_title('Camera = ' + GetCameraPosition(p) + ' M' + ', $t_\mathrm{start}$ = ' + \
                 GetTime(p) + ' M', fontsize=20, loc='left')
    
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis._axinfo['tick']['inward_factor'] = 0
    ax.xaxis._axinfo['tick']['outward_factor'] = 0.4
    ax.yaxis._axinfo['tick']['inward_factor'] = 0
    ax.yaxis._axinfo['tick']['outward_factor'] = 0.4
    ax.zaxis._axinfo['tick']['inward_factor'] = 0
    ax.zaxis._axinfo['tick']['outward_factor'] = 0.4
    ax.zaxis._axinfo['tick']['outward_factor'] = 0.4
    #ax.set_axis_off()

    plt.legend()
    #plt.tight_layout()
    plt.show()
    


In [24]:
PlotGetTrajectoriesFromH5('Data/TraceHeadOn_0_0_100_269.7/')

7 3


## Plot lensing refinement method results + final positions

In [9]:
## Plot X, Y camera data and kappas on top of it

def PlotRefinementMethodResult(p, shadow=False, zero_crossings=False):
    
    tags, x_pos, y_pos, surface = GetCameraData(p)

    fig = plt.figure(figsize=(10,8))
    
    if shadow:
        
        s = 5.0
    
        inf = GrabSurfaceIndices(surface, 1)
        aha = GrabSurfaceIndices(surface, 2)
        ahb = GrabSurfaceIndices(surface, 3)
        ahc = GrabSurfaceIndices(surface, 4)
        infty = GrabSurfaceIndices(surface, 7)
        
        #plt.scatter(x_pos[infty], y_pos[infty], s=s, c='#aeebd8', label=r'Out to $\infty$')
        #plt.scatter(x_pos[inf], y_pos[inf], s=s, c='lightblue', label=r'Out to $\infty$')
        #plt.scatter(x_pos[aha], y_pos[aha], s=s, c='#00abd1', label=r'Close to AH A')
        plt.scatter(x_pos[ahb], y_pos[ahb], s=s, c='#d10084', label=r'Close to AH B', picker = 1)
        #plt.scatter(x_pos[ahc], y_pos[ahc], s=s, c='black', label=r'Close to Final AH')
        
        #tolerance = 1 # points
        #ax.plot(range(10), 'o', picker=tolerance)
        
        def on_pick(event):
            ind = event.ind
            print(ind, x_pos[ahb][ind], y_pos[ahb][ind], tags[ahb][ind])
            print()

        fig.canvas.callbacks.connect('pick_event', on_pick)
    
    elif zero_crossings:
        
        s = 1.0
        nums = [7, 6, 5, 4, 3, 2, 1, 0]
        #nums = [4]
        cs = ['orange', 'purple', '#00eeff', 'black', 'blue', 'red', 'yellow', 'black']
        infty = GrabSurfaceIndices(surface, 7)
        indices, zero = GetGeodesicsZeroCrossings(p)
        print(max(zero))
        for num, color in zip(nums, cs):
            indx = GrabSurfaceIndices(zero, num)
            ii = list(set(indx) & set(infty))
            plt.scatter(x_pos[ii], y_pos[ii], s=s, label=str(num), color=color, picker = 1)

            def on_pick(event):
                ind = event.ind
                print(ind, x_pos[ii][ind], y_pos[ii][ind], tags[ii][ind])
                print()

            if len(nums) == 1:
                fig.canvas.callbacks.connect('pick_event', on_pick)
        
    else:
        
        tags_fs, kappas = GetFrenetSerretMaxCurvatures(p)
        kappas = np.log10(kappas)
        
        indices = GrabSurfaceIndices(surface, 7)
        ss = plt.scatter(x_pos[indices], y_pos[indices], s=20, c=kappas[indices], \
                         cmap = 'jet', vmin=2, vmax = 3.75)
        cbar = fig.colorbar(ss, fraction=0.03, pad=0.04)
        cbar.set_label(r'Max $\log_{10} \kappa$', rotation=-90, labelpad=20)
        
        a = [x for _,x in sorted(zip(kappas[indices],tags[indices]))][::-1][0:4]
        print(a)

        #a = [32584, 11037, 24611, 6013]
        #for xx, yy, tag in zip(x_pos[a], y_pos[a],tags[a]):
        #    plt.text(xx, yy, str(tag), fontsize=8)
        
    plt.title('Camera = ' + GetCameraPosition(p) + ' M' + ', $t_\mathrm{start}$ = ' + \
                 GetTime(p) + ' M', fontsize=20, loc='left')

    plt.xlabel(r'Camera X')
    plt.ylabel(r'Camera Y')
    if shadow:
        legend = plt.legend(title='Geodesic Fate', loc='upper left', fontsize=16, framealpha=1.0)
        plt.setp(legend.get_title(),fontsize=20)
        for han in legend.legendHandles:
            han._sizes = [30]
    if zero_crossings:
        legend = plt.legend(title='Number of zero crossings', ncol = 3, \
                            loc='upper left', fontsize=16, framealpha=1.0)
        plt.setp(legend.get_title(),fontsize=20)
        for han in legend.legendHandles:
            han._sizes = [30]
            
    lim = 0.25
    plt.xlim(0.5 - lim, 0.5 + lim)
    plt.ylim(0.5 - lim, 0.5 + lim)
    plt.tight_layout()
#     if shadow:
#         plt.savefig('Shadow_' + GetTime(p) + '.pdf')
#     if zero_crossings: 
#         plt.savefig('Zero_' + GetTime(p) + '.pdf')
#     else: 
#         plt.savefig('ZoomCamera_' + GetTime(p) + '.pdf')
    #plt.show()
    

In [10]:
pp_arr = [269.7]
data_locs = ['Data/TraceHeadOn_0_0_100_' + str(pp) + '/' for pp in pp_arr]
for p in data_locs:
    #PlotRefinementMethodResult(p, shadow=True)
    PlotRefinementMethodResult(p, zero_crossings=True)

7


In [ ]:
## Interpolate a quantity (like the max Frenet-Serret curvature) onto the camera data
def CameraInterpolation(p):

    file = p + 'RefinementMethodData.h5'
    f = h5py.File(file, 'r')
    data = f['LensingCore.dat']
    
    tags = data[:,0]
    print(len(tags))
    tags = tags.astype(int)
    x_pos = data[:,1]
    y_pos = data[:,2]
    surface = data[:,4]
    
    tags_fs, kappas = GetFrenetSerretMaxCurvatures(p)
    print(min(kappas), max(kappas))
    kappas = np.log10(kappas)

    fig = plt.figure(figsize=(10,8))
    
    z = np.array(kappas)
    rows = np.array(x_pos) 
    cols = np.array(y_pos) 
    
    ## Number of points to interpolate to
    YY = 1000
    XX = 1000
    ## number of contours
    N_cont = 1000 
    ## Bounds for the colobar
    cmap = 'jet'
    vmin = 3.0
    vmax = 3.6
    lim = 0.2
    ## Bounds for the figure

    
    xi = np.linspace(0.5 - lim, 0.5 + lim, XX)
    yi = np.linspace(0.5 - lim, 0.5 + lim, YY)

    GD = scipy.interpolate.griddata((rows, cols), z.ravel(),
                          (xi[None,:], yi[:,None]), method='cubic')

    CS = plt.contourf(xi,yi,GD,N_cont,cmap = cmap, vmin=vmin, vmax=vmax)

    m = plt.cm.ScalarMappable(cmap=cmap)
    m.set_array(GD)
    m.set_clim(vmin, vmax)
    cbar = plt.colorbar(m, boundaries=np.linspace(vmin, vmax, 6), fraction=0.03, pad=0.01,  orientation="vertical")
    cbar.set_label(r'Max $\log_{10} \kappa$', rotation=-90, labelpad=20)
    
    plt.xlabel(r'Camera X')
    plt.ylabel(r'Camera Y')
    plt.tight_layout()
    plt.show()